In [1]:
### Convolution neural network  image classification using keras
### Put images data within their respective category and give path of parent folder in imgpath variable below
### First step convertign normal images to raw image 
### Reading folder name 
### feature will be images raw and label will be folder name you have added your data

from PIL import Image, ImageOps
import os
import numpy as np

In [2]:
desired_size = (128, 128)
def resize_files(imgpath, mypath):
    os.makedirs(mypath,exist_ok=True)
    for path, subdirs, files in os.walk(imgpath):
        for s in subdirs:
            sub_path = os.path.join(path, s)
            thumbnail_path = os.path.join(mypath, s) 
            os.makedirs(thumbnail_path, exist_ok=True)
            subfiles = os.listdir(sub_path)

            for img_file in subfiles:
                img_file_path = os.path.join(path, s, img_file)
                new_file_name = os.path.join(thumbnail_path, img_file)

                img = Image.open(img_file_path)
                img.thumbnail(desired_size)
                w = img.size[0]
                h = img.size[1]
                topleft = (int((128-w)/2), int((128-h)/2))
                square_image = Image.new('RGB', desired_size, (255, 255, 255))
                square_image.paste(img, topleft)
                square_image.save(new_file_name, "JPEG")
                
def normalize_image(arr):
    arr = arr.astype('float')
    arr /= 255
    return arr
        
def normalize_images(imgpath, rawfilespath):
    os.makedirs(rawfilespath,exist_ok=True)
    for path, subdirs, files in os.walk(imgpath):
        for s in subdirs:
            sub_path = os.path.join(path, s)
            raw_path = os.path.join(rawfilespath, s) 
            os.makedirs(raw_path, exist_ok=True)
            subfiles = os.listdir(sub_path)
            
            for img_file in subfiles:
                img_file_path = os.path.join(path, s, img_file)
                raw_file_name = os.path.join(raw_path, img_file + '.raw')
                img = Image.open(img_file_path)
                arr = np.asarray(img)
                arr = normalize_image(arr)
                arr.tofile(raw_file_name)

In [7]:
imgpath =r'add path for dataset'

In [8]:
mypath = r'add any suitable path for thumbnail folder'

In [9]:
resize_files(imgpath, mypath)

In [11]:
imgpath = mypath
rawpath =r'create a folder and add folder location for converted thumbnails pic to raw pic'
normalize_images(imgpath, rawpath)

In [12]:
import os
import numpy as np
import pickle

features=[]
labels=[]
classes=[]
def image(imgpath):
    for path, subdirs, files in os.walk(imgpath):
        for s in subdirs:
            sub_path=os.path.join(path,s)
            subfiles=os.listdir(sub_path)
            for f in subfiles:
                labels.append(classes.index(s))
                file_path=os.path.join(sub_path, f)
                f=np.fromfile(file_path)
                f=f.reshape(128,128,3)
                features.append(f)
    return features, labels

def image_classes(imgpath):
    for path, subdirs, files in os.walk(imgpath):
        for s in subdirs:
            classes.append(s)
        return classes

pickle.dump(classes, open("classes.pkl", "wb"))

In [13]:
imgpath =r'C:\Users\Vikram\Desktop\bmo\MachineLearning_microservice\Dataset\rawfiles'
image_classes(imgpath)
features, labels=image(imgpath)

features=np.asarray(features)
labels=np.asarray(labels)

In [14]:
from keras.utils import to_categorical

def encode(data):
    print('Shape of data (BEFORE encode): %s' % str(data.shape))
    encoded = to_categorical(data)
    print('Shape of data (AFTER  encode): %s\n' % str(encoded.shape))
    return encoded

C:\Users\Vikram\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [15]:
labels = encode(labels)

Shape of data (BEFORE encode): (122,)
Shape of data (AFTER  encode): (122, 8)



In [16]:
from sklearn.cross_validation import train_test_split
X_train, X_validation, y_train, y_validation= train_test_split(features,labels, test_size = 0.25, random_state = 0)

C:\Users\Vikram\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [19]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Convolution2D(32, (3, 3) , input_shape = (128, 128, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Convolution2D(32, 3, 3, activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(output_dim = 128, activation = 'relu'))

#o/p layer
classifier.add(Dense(output_dim = 8, activation = 'softmax'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

C:\Users\Vikram\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
C:\Users\Vikram\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
C:\Users\Vikram\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=8)`


In [ ]:
classifier.fit(X_train, y_train, batch_size = 25, nb_epoch = 200)

In [ ]:
import cv2

def checking_classification_bill():
    img=cv2.imread('Path of any image realted to your dataset to predict')
    img=cv2.resize(img, (128,128))
    img=img.reshape(1,128,128,3)
    pred = classifier.predict(img)
    return pred

In [ ]:
y_pred = (checking_classification_bill() > 0.5)
print(y_pred)

### Which class above image belong
print(classes)